# SQA_GAN

In [1]:
# Main file for ISING GAN
# Author Pengyuan (Bill) Zhai, Aug 2020
import numpy as np
import time
import os

import tensorflow as tf # SQA_GAN uses the tensorflow.nn implementation instead of tf v1
#from tensorflow.examples.tutorials.mnist import input_data

from matplotlib import pyplot as plt
import math
%matplotlib inline
import datetime
from sklearn.metrics import confusion_matrix
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10218422403635597915
]


In [2]:
tf.debugging.set_log_device_placement(True)
with tf.device('/CPU:0'):
    # Create some tensors
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)

    print(c)

Executing op _MklMatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


# Initialization

In [3]:
tf.debugging.set_log_device_placement(True)
# Keep all Legacy variables as it is
task_name = "ISING_32_32"
npSeed = 188
np.random.seed(npSeed)
x_height, x_width = [32, 32]
num_channels = 40 # The number of Trotter slices
num_classes = 1 # Once "class" since I am using WGAN (1 output node for the critic score)
latent_size = 100
#labeled_rate = 1.00 # Legacy from BSSGAN 1.0, 0.5, 0.1 this limits the knowledge base of the learning
#unlabeled_supp_rate = 0 # Legacy from BSSGAN: percentage of unlabeled data to be supplemented to the learning
#c_ul = 0 # Legacy code from BSSGAN: class portion of unlabeled data
# task_path = 'numpyData/BINARY_CR_16_1_split_128_128'

task_path = 'numpyData/'+task_name
# Legacy Code: keep as it is: all data (labeled and unlabeled) by class [[all data goes here]]
# Legacy Code: I'm not using real class labels here but I will keep them here for later just in case
train_data_by_class, test_data_by_class, train_label_by_class, test_label_by_class = [], [], [], []

# Note: for a certain class, the temperatures has shape [batch_size]: [temp0, temp1,...,]
train_temp_by_class = [] # Extra information (temperature) for CGAN
test_temp_by_class = [] # Extra information (temperature) for CGAN

# train_mask_by_class = [] # Commented out, Legacy code
#unlabeled_indices_by_class = []
#labeled_indices_by_class = [] # Legacy: Marked for labeled data selection to batch

#unlabeled_data = np.zeros((0, x_width, x_height, num_channels)) # Legacy: In order to randomly select from
# all unlabeled data without considering classes

# Legacy code: export baseline data
#labeled_data_baseline = np.zeros((0, x_width, x_height, num_channels)) # In order to export 0.25 labeled 
# Load real data:
for i in range(num_classes):
    train_data_by_class.append(np.load(task_path+'/class_'+str(i)+'/trainX.npy'))
    #train_label_by_class.append(np.load(task_path+'/class_'+str(i)+'/trainy.npy'))
    train_temp_by_class.append(np.load(task_path+'/class_'+str(i)+'/train_temp.npy'))
    
    test_data_by_class.append(np.load(task_path+'/class_'+str(i)+'/testX.npy'))
    #test_label_by_class.append(np.load(task_path+'/class_'+str(i)+'/testy.npy'))
    test_temp_by_class.append(np.load(task_path+'/class_'+str(i)+'/test_temp.npy'))
    print("clas+ "+str(i)+'has '+str(len(train_data_by_class[i])))

    
# Legacy code: semi-supervised. Might be useful in the future
    # Select data as unlabeled:
# for i in range(num_classes):
#     numInClass = len(train_label_by_class[i])
#     #mask =  np.concatenate((np.ones(int(numInClass * labeled_rate), dtype='int'), np.zeros(numInClass - int(numInClass * labeled_rate), dtype='int')), axis=0)
#     num_unlabeled = int(numInClass-numInClass*labeled_rate)
#     train_data_class = train_data_by_class[i]
#     train_label_class = train_data_by_class[i]
#     indices = [i for i in range(numInClass)]
#     np.random.shuffle(indices)
#     unlabeled_is, labeled_is = np.split(indices,[num_unlabeled]) # Unlabeled and labeled indices
    
#     # Concatenate image data array
#     unlabeled_data = np.concatenate((unlabeled_data, train_data_by_class[i][unlabeled_is]))
#     labeled_data_baseline = np.concatenate((labeled_data_baseline, train_data_by_class[i][labeled_is]))
    
#     #unlabeled_indices_by_class.append(unlabeled_is)
#     labeled_indices_by_class.append(labeled_is)

#np.save(unlabeled_data, 'numpyData/0.25_labeled_data')
    
numTrain = sum([len(c) for c in train_data_by_class])
numTest = sum([len(c) for c in test_data_by_class])
log_path = './ISING_GAN_log_' + task_name + '.csv' # Don't worry about log_path, it is named after the task
#log_path_baseline = './baseline_log.csv'
model_path ='./savedModels/GAN'
#baseline_path = './savedModels/baseline'

clas+ 0has 18000


In [4]:
train_data_by_class[0].shape

(18000, 32, 32, 40)

# Prepare Data

In [5]:
def normalize(x):
    # normalize data
#     x /= 255.0
    x = x
    return x.reshape((-1, x_height, x_width, num_channels)) #x is 4 dimensional-- (num_images, height, width, depth)

In [6]:
#save masked/labeled image to numpy array
# def save_masked(toFolder):
#     for i in range(num_classes):
#         if not os.path.exists('numpyData/'+toFolder+'/class_'+str(i)):
#             os.makedirs('numpyData/'+toFolder+'/class_'+str(i))
#         np.save('numpyData/'+toFolder+'/class_'+str(i)+'/trainX', train_data_by_class[i][train_mask_by_class[i].astype(bool)])
#         np.save('numpyData/'+toFolder+'/class_'+str(i)+'/trainy', train_label_by_class[i][train_mask_by_class[i].astype(bool)])

In [7]:
#save_masked("labeled_for_cnn_seed_"+str(npSeed)+'_rate_'+str(labeled_rate))

In [8]:
#shuffle data array, labels array, and labeledMask array, each image's properties should remain consistant (labeled/unlabeled)
#during the whole experiment. Instead of assigning labeled mask per next batch, it should be globally defined
#prior to running the experiment --BZ, August, 2019
def shuffle_data(data, labels, labeledMask):#all arrays here are row vectors? labels and data are columnwise
    #np.random.seed(123)#for debugging purpose
    indices = np.arange(labels.shape[0]) #index sequence whose length = len(labels)
    np.random.shuffle(indices) #In place
    shuffled_indices = indices #Useless assignment for clarity- shuffle is in place
    if labeledMask is None:#for cases used by get_test_batch() function--BZ, August, 2019
        return data[shuffled_indices], labels[shuffled_indices]
    else:
        return data[shuffled_indices], labels[shuffled_indices], labeledMask[shuffled_indices]


In [9]:
#new batch functions:--BZ, August, 2019
#because labeled mask should stay with the same images after each shuffle (which happens at the start of each new epoch),
#get_batch() and get_labeled_mask() should be merged into one function
# def get_training_batch_and_labeled_mask(XTrain, yTrain, labeledMask, batchSize):
#     #first shuffle the indices:
#     XTrainRandom, yTrainRandom, labeledMaskRandom = shuffle_data(XTrain, yTrain, labeledMask);
#     #a generator that slices and returns batchSize of XTrain and yTrain instances from top down
#     counter = 0;
#     while True:
#         if counter >= len(yTrain):
#             break;
#         returnXTrain = XTrainRandom[counter:counter + batchSize];
#         returnYTrain = yTrainRandom[counter:counter + batchSize];
#         returnLabeledMask = labeledMaskRandom[counter:counter + batchSize];
#         counter = counter + batchSize;
#         yield returnXTrain, returnYTrain, returnLabeledMask
# def get_test_batch(XTest, yTest, batchSize): #essentially the same function as above, restated here for explicity
#     #first shuffle the indices:
#     XTestRandom, yTestRandom = shuffle_data(XTest, yTest, None);
#     #a generator that slices and returns batchSize of XTrain and yTrain instances from top down
#     counter = 0;
#     while True:
#         if counter >= len(yTest):
#             break;
#         returnXTest = XTestRandom[counter:counter + batchSize];
#         returnYTest = yTestRandom[counter:counter + batchSize];
#         counter = counter + batchSize;
#         yield returnXTest, returnYTest
    
# def get_balance_train_batch(train_data_by_class, train_label_by_class, train_mask_by_class, batchSize):
#     numEachClass = int(np.floor(batchSize / num_classes))
#     returnData = np.zeros((0, x_width, x_height, num_channels))
#     returnLabel = np.zeros((0, num_classes))
#     returnLabeledMask = np.zeros((0))
#     for i in range(num_classes):
#         train_data = train_data_by_class[i]
#         train_label = train_label_by_class[i]
#         train_mask = train_mask_by_class[i]
#         indices = [v for v in range(len(train_label))]
#         np.random.shuffle(indices)
#         selectIndices = indices[0:numEachClass]
#         returnData = np.concatenate((returnData, train_data[selectIndices]))
#         returnLabel = np.concatenate((returnLabel, train_label[selectIndices]))
#         returnLabeledMask = np.concatenate((returnLabeledMask, train_mask[selectIndices]))
#     return returnData, returnLabel, returnLabeledMask

# def get_balance_train_batch_2(train_data_by_class, train_label_by_class, train_mask_by_class, batchSize):
#     numEachClass = int(batchSize / num_classes)
#     numEachClass_unlabeled_labeled = [numEachClass - int(numEachClass*labeled_rate), int(numEachClass*labeled_rate)]
# #     print(numEachClass)
# #     print(numEachClass_unlabeled_labeled)
#     returnData = np.zeros((0, x_width, x_height, num_channels))
#     returnLabel = np.zeros((0, num_classes))
#     returnLabeledMask = np.zeros((0))
#     for i in range(num_classes):
#         for j in [0,1]: #for unlabeled and labeled
#             indices = []
#             for k in range(len(train_mask_by_class[i])):
#                 if train_mask_by_class[i][k] == j:
#                     indices.append(k)
# #             print('label status_'+str(j))
# #             print(indices)
#             np.random.shuffle(indices)
#             selectIndices = indices[0:numEachClass_unlabeled_labeled[j]]
# #             print('selected')
# #             print(selectIndices)
#             train_data = train_data_by_class[i]
#             train_label = train_label_by_class[i]
#             train_mask = train_mask_by_class[i]
            
#             returnData = np.concatenate((returnData, train_data[selectIndices]))
#             returnLabel = np.concatenate((returnLabel, train_label[selectIndices]))
#             returnLabeledMask = np.concatenate((returnLabeledMask, train_mask[selectIndices]))
#     return returnData, returnLabel, returnLabeledMask

# Legacy:
# Input: c_ul is the relative portion ratio related to each class in a batch
# def get_balance_train_batch_3(train_data_by_class, train_temp_by_class, batchSize, c_ul):
#     numEachPortion = int(batchSize/(num_classes+1+c_ul)) # One portion of fake,c_ul portion of unlabeled
#     returnData = np.zeros((0, x_width, x_height, num_channels))
#     returnLabel = np.zeros((0, num_classes))
#     returnLabeledMask = np.zeros((0))
#     # First load labeled Data (mask is 1)
#     for i in range(num_classes):
#         labeled_is = np.random.choice(labeled_indices_by_class[i], numEachPortion)
#         returnData = np.concatenate((returnData, train_data_by_class[i][labeled_is]))
#         returnLabel = np.concatenate((returnLabel, train_label_by_class[i][labeled_is]))
#         returnLabeledMask = np.concatenate((returnLabeledMask, np.ones(numEachPortion))) # 1 for labeled
#     # Then load unlabeled Data (mask is 0)
#     if len(unlabeled_data) is not 0:
#         num_unlabeled = numEachPortion * c_ul
#         unlabeled_is = np.random.choice([i for i in range(len(unlabeled_data))], num_unlabeled)
#         returnData = np.concatenate((returnData, unlabeled_data[unlabeled_is]))
#         returnLabel = np.concatenate((returnLabel, np.zeros((num_unlabeled, num_classes))))
#         returnLabeledMask = np.concatenate((returnLabeledMask, np.zeros(num_unlabeled))) # 0 for unlabeled
#     return returnData, returnLabel, returnLabeledMask


In [10]:
def get_train_batch(train_data_by_class, train_temp_by_class, batch_size):
    return get_test_batch(train_data_by_class, train_temp_by_class, batch_size)
        

def get_test_batch(test_data_by_class, test_temp_by_class, batchSize):
    XTest = np.zeros((0, x_width, x_height, num_channels))
    yTest = np.zeros((0, 1)) # For temperatures
    for i in range(num_classes):
        XTest = np.concatenate((XTest, test_data_by_class[i]))
        yTest = np.concatenate((yTest, test_temp_by_class[i]))
    #first shuffle the indices:
    XTestRandom, yTestRandom = shuffle_data(XTest, yTest, None);
    #a generator that slices and returns batchSize of XTrain and yTrain instances from top down
    counter = 0;
    while True:
        if counter >= len(yTest):
            break;
        returnXTest = XTestRandom[counter:counter + batchSize];
        returnYTest = yTestRandom[counter:counter + batchSize];
        counter = counter + batchSize;
        yield returnXTest, returnYTest

In [11]:
for x, temp in get_train_batch(test_data_by_class, test_temp_by_class, 1):
    print(x[0][0][0][0])
    print(temp)

1.0
[[0.22846143]]
1.0
[[1.90269514]]
1.0
[[2.98949948]]
1.0
[[0.58427922]]
1.0
[[0.16110806]]
1.0
[[1.42222112]]
1.0
[[0.9018451]]
1.0
[[1.14170709]]
1.0
[[2.36066804]]
1.0
[[2.53227662]]
1.0
[[2.66173309]]
1.0
[[1.06295315]]
1.0
[[2.48712436]]
1.0
[[0.99004951]]
1.0
[[2.03905196]]
1.0
[[0.58862944]]
1.0
[[2.11315566]]
1.0
[[1.55467774]]
1.0
[[0.28201411]]
1.0
[[1.82514126]]
1.0
[[0.98419922]]
1.0
[[2.07550378]]
1.0
[[1.55737787]]
1.0
[[2.27651383]]
1.0
[[1.4118706]]
1.0
[[2.30426522]]
1.0
[[2.80769039]]
1.0
[[2.49672484]]
1.0
[[0.85609281]]
1.0
[[0.43742188]]
1.0
[[2.37566879]]
1.0
[[2.28926447]]
1.0
[[2.63578179]]
1.0
[[2.33756688]]
1.0
[[2.84654233]]
1.0
[[2.62363118]]
1.0
[[2.35751788]]
1.0
[[2.35676784]]
1.0
[[1.46512326]]
1.0
[[0.25741288]]
1.0
[[2.04070204]]
1.0
[[1.85409271]]
1.0
[[0.57887895]]
1.0
[[1.17845893]]
1.0
[[1.16510826]]
1.0
[[0.13230662]]
1.0
[[2.92064603]]
1.0
[[1.72163609]]
1.0
[[2.60353018]]
1.0
[[1.46977349]]
1.0
[[1.81374069]]
1.0
[[2.16430822]]
1.0
[[2.394569

1.0
[[2.73718686]]
1.0
[[2.11420571]]
1.0
[[0.04800241]]
1.0
[[0.88159409]]
1.0
[[0.13830692]]
1.0
[[2.05735287]]
1.0
[[2.97089855]]
1.0
[[0.87469374]]
1.0
[[0.30256514]]
1.0
[[2.20541027]]
1.0
[[0.24196211]]
1.0
[[0.7357868]]
1.0
[[1.59472974]]
1.0
[[0.14220712]]
1.0
[[0.25441273]]
1.0
[[0.78198911]]
1.0
[[1.5819791]]
1.0
[[1.15340768]]
1.0
[[2.94359718]]
1.0
[[2.84774239]]
1.0
[[2.08735437]]
1.0
[[2.4319216]]
1.0
[[1.83309166]]
1.0
[[2.62318116]]
1.0
[[2.54022701]]
1.0
[[0.14895746]]
1.0
[[0.41507076]]
1.0
[[1.47892395]]
1.0
[[0.02295116]]
1.0
[[1.32276614]]
1.0
[[1.33071654]]
1.0
[[1.26201311]]
1.0
[[0.56402821]]
1.0
[[2.50572529]]
1.0
[[1.84674234]]
1.0
[[2.03575179]]
1.0
[[2.25251263]]
1.0
[[0.21466074]]
1.0
[[1.59998]]
1.0
[[1.34571729]]
1.0
[[1.3518676]]
1.0
[[2.29226462]]
1.0
[[0.94099706]]
1.0
[[0.05370269]]
1.0
[[0.56642833]]
1.0
[[2.36486825]]
1.0
[[2.79418971]]
1.0
[[0.58262914]]
1.0
[[0.95224762]]
1.0
[[1.81914096]]
1.0
[[2.95769789]]
1.0
[[1.25556278]]
1.0
[[0.77703886]]


[[0.47747388]]
1.0
[[0.059853]]
1.0
[[2.82554128]]
1.0
[[2.39981999]]
1.0
[[1.72448623]]
1.0
[[2.97134857]]
1.0
[[0.54962749]]
1.0
[[2.79913996]]
1.0
[[0.17520877]]
1.0
[[1.04090205]]
1.0
[[2.71333567]]
1.0
[[1.18685935]]
1.0
[[2.45022251]]
1.0
[[2.85869294]]
1.0
[[2.6119306]]
1.0
[[0.34051703]]
1.0
[[2.56467824]]
1.0
[[2.16850843]]
1.0
[[2.04535227]]
1.0
[[2.94164708]]
1.0
[[1.72373619]]
1.0
[[2.19040952]]
1.0
[[0.24736238]]
1.0
[[2.47497375]]
1.0
[[2.02225112]]
1.0
[[0.40562029]]
1.0
[[2.1079054]]
1.0
[[2.88554428]]
1.0
[[2.45697285]]
1.0
[[2.21681084]]
1.0
[[0.19890995]]
1.0
[[1.97979899]]
1.0
[[2.5679784]]
1.0
[[0.0478524]]
1.0
[[0.30601531]]
1.0
[[2.30846543]]
1.0
[[0.45347268]]
1.0
[[0.17295866]]
1.0
[[2.86139307]]
1.0
[[0.81934097]]
1.0
[[1.43917196]]
1.0
[[2.67433372]]
1.0
[[2.81669084]]
1.0
[[2.91929597]]
1.0
[[1.4238712]]
1.0
[[2.46582329]]
1.0
[[1.05695285]]
1.0
[[0.6498325]]
1.0
[[1.88229412]]
1.0
[[0.3678184]]
1.0
[[2.06470324]]
1.0
[[1.64948248]]
1.0
[[0.45512276]]
1.0
[[

[[1.20501026]]
1.0
[[2.70733537]]
1.0
[[0.84889245]]
1.0
[[0.65523277]]
1.0
[[1.08620432]]
1.0
[[1.17755888]]
1.0
[[1.65083255]]
1.0
[[1.44922247]]
1.0
[[1.66943348]]
1.0
[[2.48232412]]
1.0
[[1.38351918]]
1.0
[[2.03230162]]
1.0
[[2.66818341]]
1.0
[[0.87709386]]
1.0
[[2.66713336]]
1.0
[[1.6679334]]
1.0
[[0.64233212]]
1.0
[[2.27666384]]
1.0
[[2.64823241]]
1.0
[[0.28171409]]
1.0
[[0.63483175]]
1.0
[[1.69103456]]
1.0
[[2.13565679]]
1.0
[[1.85979299]]
1.0
[[0.79068954]]
1.0
[[2.95469774]]
1.0
[[2.75923796]]
1.0
[[1.13240663]]
1.0
[[2.13820691]]
1.0
[[0.10020502]]
1.0
[[0.11850593]]
1.0
[[0.37306866]]
1.0
[[2.99384969]]
1.0
[[2.23406171]]
1.0
[[0.53402671]]
1.0
[[0.77538878]]
1.0
[[2.35421771]]
1.0
[[1.37976899]]
1.0
[[2.93879694]]
1.0
[[0.91624582]]
1.0
[[0.63543178]]
1.0
[[1.06700336]]
1.0
[[0.35116757]]
1.0
[[1.02170109]]
1.0
[[1.19825992]]
1.0
[[1.11470574]]
1.0
[[2.0078504]]
1.0
[[0.19050953]]
1.0
[[0.58712936]]
1.0
[[1.49827492]]
1.0
[[0.00660034]]
1.0
[[2.78098905]]
1.0
[[1.13225662]]

In [12]:
np.sum(train_data_by_class[0][0] == train_data_by_class[0][100])

40949

In [13]:
#data, label, mask = get_balance_train_batch_3(train_data_by_class, train_label_by_class, train_mask_by_class, 60, 1)

In [14]:
# data, label, mask = get_balance_train_batch_2(train_data_by_class, train_label_by_class, train_mask_by_class, 60)

In [15]:
# Calculate average magnetization matrices
# Input x has shape: [batch_size, 32, 32, 1]
def get_avg_mag(x):
    batch_size = x.get_shape()[0]
    # Create average magnetization matrix
    avg_mag = tf.math.reduce_mean(x, axis=[1, 2, 3]) # Take mean of each graph
    assert avg_mag.get_shape().as_list() == [batch_size]
    concat_array = []
    for i in range(batch_size):
        fill_tensor = tf.fill(dims=[32, 32, 1], value=avg_mag[i]) # Fill 32*32*1 tensorf with avg mag value
        fil_tensor_expand = tf.expand_dims(fill_tensor, axis=0) # Expand tensor dimension for batch size
        concat_array.append(fil_tensor_expand)
    return_tensor = tf.concat(concat_array, axis=0) 
    assert return_tensor.get_shape() == [bath_size, 32, 32, 1]
    return return_tensor

# Build Model

In [16]:
def spectral_norm(w, iteration=1):
    w_shape = w.shape.as_list()
    w = tf.reshape(w, [-1, w_shape[-1]])

    #u = tf.Variable("u", [1, w_shape[-1]], initializer=tf.random_normal_initializer(), trainable=False)
    u = tf.Variable(tf.random.normal(shape=[1, w_shape[-1]]), trainable=False)
    u_hat = u
    v_hat = None
    for i in range(iteration):
        v_ = tf.matmul(u_hat, tf.transpose(w))
        v_hat = tf.nn.l2_normalize(v_)

        u_ = tf.matmul(v_hat, w)
        u_hat = tf.nn.l2_normalize(u_)

    u_hat = tf.stop_gradient(u_hat)
    v_hat = tf.stop_gradient(v_hat)

    sigma = tf.matmul(tf.matmul(v_hat, w), tf.transpose(u_hat))

    with tf.control_dependencies([u.assign(u_hat)]):
        w_norm = w / sigma
        w_norm = tf.reshape(w_norm, w_shape)


    return w_norm

In [17]:
# The discriminator/critic
# Note: temperatures tensor has shape [batch_size]: [temp0, temp1, temp2,...]
def D(x, temperatures, dropout_rate, is_training, reuse = True, print_summary = True):
#def D(x, temperature, dropout_rate, is_training, reuse = True, print_summary = True):
    # discriminator (x -> n + 1 class)

    with tf.compat.v1.variable_scope('Discriminator', reuse = reuse) as scope:
        batch_size = x.get_shape().as_list()[0] # Get the batch size
        
        # Weight matrix for fully connected layer--alternative version
        #W_fc_input = tf.Variable(tf.truncated_normal([1, 32*32*1], stddev=0.1), name='W_fc_input')
        #b_fc_input = tf.Variable(tf.constant(0.1, shape=[32*32*1]), name='b_fc_input')
        #temp_fc = tf.nn.xw_plus_b(temperatures, W_fc_input, b_fc_input, name='temp_fc')
        #temp_reshaped = tf.reshape(temp_fc, [-1, 32, 32, 1])
        
        # Concatenate the avg_mag tensor with the input
        #concat = tf.concat([x, temp_reshaped], axis=3)
        #assert concat.shape.as_list()[1:] == [32, 32, 2]
        
        # layer1 - do not use Batch Normalization on the first layer of Discriminator
        # The W filter has shape: [height, width, input_channels, output_channels]
        w_conv1 = tf.Variable(tf.random.truncated_normal([3, 3, num_channels, 32], stddev=0.1), name='w_conv1')
        #conv1 = tf.layers.conv2d(concat, 32, [3, 3],
                                 #strides = [2, 2],
                                 #padding = 'same')
        conv1 = tf.nn.conv2d(x, spectral_norm(w_conv1), strides=[2,2], padding='SAME', name='conv1')
        b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]), name='b_conv1') # Bias
        #lrelu1 = tf.maximum(0.2 * conv1, conv1) #leaky relu
        lrelu1 = tf.nn.leaky_relu(tf.nn.bias_add(conv1, b_conv1), alpha=0.2, name='lrelu1')
        
        dropout1 = tf.nn.dropout(lrelu1, dropout_rate)

        # layer2
#         conv2 = tf.layers.conv2d(dropout1, 64, [3, 3],
#                                  strides = [2, 2],
#                                  padding = 'same')
#         batch_norm2 = tf.layers.batch_normalization(conv2, training = is_training, momentum=0.8)
#         lrelu2 = tf.maximum(0.2 * batch_norm2, batch_norm2)
#         dropout2 = tf.layers.dropout(lrelu2, dropout_rate)
        w_conv2 = tf.Variable(tf.random.truncated_normal([3, 3, 32, 64], stddev=0.1), name='w_conv2')
        conv2 = tf.nn.conv2d(dropout1, spectral_norm(w_conv2), strides=[2,2], padding='SAME', name='conv2')
        b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]), name='b_conv2') # Bias
        lrelu2 = tf.nn.leaky_relu(tf.nn.bias_add(conv2, b_conv2), alpha=0.2, name='lrelu2')
        dropout2 = tf.nn.dropout(lrelu2, dropout_rate)

        # layer3
#         conv3 = tf.layers.conv2d(lrelu2, 64, [3, 3],
#                                  strides = [1, 1],
#                                  padding = 'same')
#         batch_norm3 = tf.layers.batch_normalization(conv3, training = is_training)
#         lrelu3 = tf.maximum(0.2 * batch_norm3, batch_norm3)
#         dropout3 = tf.layers.dropout(lrelu3, dropout_rate)
        w_conv3 = tf.Variable(tf.random.truncated_normal([3, 3, 64, 64], stddev=0.1), name='w_conv3')
        conv3 = tf.nn.conv2d(dropout2, spectral_norm(w_conv3), strides=[1,1], padding='SAME', name='conv3')
        b_conv3 = tf.Variable(tf.constant(0.1, shape=[64]), name='b_conv3') # Bias
        lrelu3 = tf.nn.leaky_relu(tf.nn.bias_add(conv3, b_conv3), alpha=0.2, name='lrelu3')
        dropout3 = tf.nn.dropout(lrelu3, dropout_rate)
        #dropout3 = tf.layers.dropout(lrelu3, dropout_rate)
        #dropout3 = tf.layers.dropout(lrelu2, dropout_rate)
        # layer 4
        #conv4 = tf.layers.conv2d(dropout3, 256, [3, 3],
                                #strides = [1, 1],
                                #padding = 'same')
        # do not use batch_normalization on this layer - next layer, "flatten5",
        # will be used for "Feature Matching"
        #lrelu4 = tf.maximum(0.2 * conv4, conv4)

        # layer 5
        flatten_length = dropout3.get_shape().as_list()[1] * \
                         dropout3.get_shape().as_list()[2] * dropout3.get_shape().as_list()[3]
        flatten5 = tf.reshape(dropout3, (-1, flatten_length)) # used for "Feature Matching" 
        
        w_flatten5 = tf.Variable(tf.random.truncated_normal([flatten_length, 2], stddev=0.1), name='w_flatten5')
        b_flatten5 = tf.Variable(tf.constant(0.1, shape=[2]), name='b_flatten5')
        fc5 = tf.math.add(tf.linalg.matmul(flatten5, w_flatten5), b_flatten5, name='fc5')
        #fc5 = tf.nn.xw_plus_b(flatten, w_flatten5, b_flatten5, name='fc5')
        #fc5 = tf.layers.dense(flatten5, (1)) # WGAN only needs one output node for the critic score
        #output = tf.nn.softmax(fc5, name="D_output")
        assert fc5.get_shape()[1:] == [2]
        output_classification_prob = fc5[:,0:1] # A probability score (0 to 1) enforced lip-1 by spect-norm
        output_regression_label = tf.nn.leaky_relu(fc5[:,1:2], alpha=0.2) # Continuous label regression, leaky relu activation
        print('output_classification_prob.get_shape() is')
        print(output_classification_prob.get_shape())
        assert output_classification_prob.get_shape()[1:] == [1]
        assert output_regression_label.get_shape()[1:] == [1]
        
        if print_summary:
            print('Discriminator summary:\n x: %s\n' \
                  ' D1: %s\n D2: %s\n D3: %s\n D4: %s\n' %(x.get_shape(), 
                                                           dropout1.get_shape(),
                                                           lrelu2.get_shape(), 
                                                           dropout3.get_shape(),
                                                           lrelu4.get_shape()))
        #return flatten5, fc5, output
        #debug: return each layer's output --BZ Nov 28, 2019
        return flatten5, fc5, output_classification_prob, output_regression_label

In [18]:
testA = np.zeros([5,2])
testA[:,0:2]

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [19]:
#def G(z, is_training, reuse = False, print_summary = False):
# Input: temperatures is a list of temperatures with size batch_size, where the temperature values match that
# from the real data in the batch (in value and in order)
# Note: temperatures tensor has shape [batch_size]: [temp0, temp1, temp2]
def G(z, temperatures, is_training, reuse = False, print_summary = False): # Added: temperature
    # generator (z -> x)

    with tf.compat.v1.variable_scope('Generator', reuse = reuse) as scope:
        batch_size = z.get_shape().as_list()[0] # Get the batch size
        # Create temperature "embedding":
        #temp_50 = tf.repeat(temperatures, repeats=[50], axis=1)
        temp_fc = tf.compat.v1.layers.dense(temperatures, 8*8*1)
        temp_reshaped = tf.compat.v1.reshape(temp_fc, [-1, 8, 8, 1])
        
        
        #z has shape [batch_size, 100]
        fc1 = tf.compat.v1.layers.dense(z, 8*8*128)
        # layer 0
        z_reshaped = tf.compat.v1.reshape(fc1, [-1, 8, 8, 128])

        
        #Concatenate the noise matrix and the temperature matrix as per CGAN specs
        concat = tf.compat.v1.concat([z_reshaped, temp_reshaped], axis=3)
        assert concat.shape.as_list()[1:] == [8, 8, 129]
        
        # layer 1
        deconv1 = tf.compat.v1.layers.conv2d_transpose(concat,
                                             filters = 129,
                                             kernel_size = [3, 3],
                                             strides = [2, 2],
                                             padding = 'same')
        batch_norm1 = tf.compat.v1.layers.batch_normalization(deconv1, training = is_training, momentum=0.8)
        relu1 = tf.compat.v1.nn.relu(batch_norm1)
        #64*64*64
        # layer 2
        deconv2 = tf.compat.v1.layers.conv2d_transpose(relu1,
                                             filters = 64,
                                             kernel_size = [3, 3],
                                             strides = [2, 2],
                                             padding = 'same')
        batch_norm2 = tf.compat.v1.layers.batch_normalization(deconv2, training = is_training, momentum=0.8)
        relu2 = tf.compat.v1.nn.relu(batch_norm2)
        #128*128*3
        # layer 3
        #deconv3 = tf.layers.conv2d_transpose(relu2,
                                             #filters = 64,
                                             #kernel_size = [3, 3],
                                             #strides = [1, 1],
                                             #padding = 'same')
        #batch_norm3 = tf.layers.batch_normalization(deconv3, training = is_training)
        #relu3 = tf.nn.relu(batch_norm3)

        # layer 4 - do not use Batch Normalization on the last layer of Generator
        deconv4 = tf.compat.v1.layers.conv2d_transpose(relu2,
                                             filters = num_channels,
                                             kernel_size = [3, 3],
                                             strides = [1, 1],
                                             padding = 'same')
        tanh4 = tf.compat.v1.tanh(deconv4, name="G_output")

        assert tanh4.get_shape()[1:] == [x_height, x_width, num_channels]
        if print_summary:
            print('Generator summary:\n z: %s\n' \
                  ' G0: %s\n G1: %s\n G2: %s\n G3: %s\n G4: %s\n' %(z.get_shape(),
                                                                    z_reshaped.get_shape(),
                                                                    relu1.get_shape(),
                                                                    relu2.get_shape(),
                                                                    relu3.get_shape(),
                                                                    tanh4.get_shape()))
        return tanh4 # The fake image

In [20]:
#build model for each batch using D() and G() functions
#def build_model(x_real, z, label, dropout_rate, is_training, print_summary = False):
def build_model(x_real, z, temperatures, dropout_rate, is_training, print_summary = False):
    # build model
    #Discriminator on real data (labeled and unlabeled)  flatten5, fc5, output
    # Due to one output node, the logit is kinda useless
    D_real_features, D_real_logit, D_real_prob, D_real_regression = D(x_real, temperatures, dropout_rate, is_training,
                                                   reuse = False, print_summary = print_summary)
    #generate fake images
    x_fake = G(z, temperatures, is_training, reuse = False, print_summary = print_summary)
    #Discriminator for fake images
    D_fake_features, D_fake_logit, D_fake_prob, D_fake_regression= D(x_fake, temperatures, dropout_rate, is_training,
                                                   reuse = True, print_summary = print_summary)

    return D_real_features, D_real_logit, D_real_prob, D_real_regression, D_fake_features, D_fake_logit, D_fake_prob, D_fake_regression, x_fake

In [21]:
#def loss_accuracy(D_real_features, D_real_logit, D_real_prob, D_fake_features,
                  #D_fake_logit, D_fake_prob, extended_label, labeled_mask):
# Note: real_imgs and fake_imgs have the same temperature/goal temperature
# Note: input "label" is only for real classes: 0 for real, 1 for fake
def loss_accuracy(real_imgs, fake_imgs, D_real_features, D_real_logit, D_real_prob, D_real_regression, D_fake_features,
                  D_fake_logit, D_fake_prob, D_fake_regression, real_labels):
    epsilon = 1e-8 # used to avoid NAN loss
    prob_real_be_real = D_real_prob + epsilon
    prob_fake_be_real = D_fake_prob + epsilon 
    # *** Discriminator loss ***
    # For SQA-GAN, the discriminiator has L_regression and L_hinge, where L_regression is for 
    # laebeld data (continuous label)--which is supervised
    # and L_hinge is unsupervised loss
    print('real_imgs')
    batch_size = tf.shape(real_imgs)[0]
    print('batch_size is')
    print(batch_size)
    # L_regression_error: Use mean here because batch sizes are not guaranteed to agree.
    D_L_regression_error = tf.reduce_mean(tf.square(D_real_regression - real_labels), axis=0)
    # First part of L_hinge
    D_L_hinge_real_be_real = -1.0*tf.reduce_mean(tf.math.minimum(tf.constant(0.0, shape=[1]), \
                    tf.constant(-1.0, shape=[1])+prob_real_be_real), axis=0)
    # Second part of L_hinge
    D_L_hinge_fake_be_fake = -1.0*tf.reduce_mean(tf.math.minimum(tf.constant(0.0, shape=[1]), \
                    tf.constant(-1.0, shape=[1])-1*prob_fake_be_real), axis=0)
    D_L_hinge = D_L_hinge_real_be_real + D_L_hinge_fake_be_fake
    
    D_L = D_L_hinge + 100*D_L_regression_error
    #prob_real_be_real = D_real_prob + epsilon
    #tmp_log = tf.log(prob_real_be_real)
    #D_L_unsupervised1 = -1 * tf.reduce_mean(tmp_log)
    #D_L_unsupervised1 = -1 * tf.reduce_mean(prob_real_be_real, axis=0)

    # data is fake
    #prob_fake_be_real = D_fake_prob + epsilon # The "real class" probability score on fake data
    #tmp_log = tf.log(prob_fake_be_fake)
    #D_L_unsupervised2 = -1 * tf.reduce_mean(tmp_log)
    #D_L_unsupervised2 = tf.reduce_mean(prob_fake_be_real, axis=0)
    
    #D_L = D_L_unsupervised2 + D_L_unsupervised1

    # *** Generator loss ***
    # Four parts to generator loss: 1. Fool the discriminator, 2. Fool the temperature regressor, 3. Feature mapping
    # 4. Avg_mag.
    # fake data is mistaken to be real (Fool the discriminator)
    #tmp_log =  tf.log(prob_fake_be_real)
    G_L1 = -1 * tf.reduce_mean(prob_fake_be_real, axis=0)

    # Fool the discriminator regrossor
    G_L2 = tf.reduce_mean(tf.square(real_labels - D_fake_regression), axis=0)
    
    # Feature Maching
    tmp1 = tf.reduce_mean(D_real_features, axis = 0)
    tmp2 = tf.reduce_mean(D_fake_features, axis = 0)
    G_L3 = tf.reduce_mean(tf.square(tmp1 - tmp2)) # Mean or sum, both are ok I think
    
    # Avg_mag
    avg_mag_real = tf.math.reduce_mean(real_imgs, axis=[1, 2, 3]) # Take mean of each graph
    avg_mag_fake = tf.math.reduce_mean(fake_imgs, axis=[1, 2, 3])
    G_L4 = tf.reduce_mean(tf.square(avg_mag_real-avg_mag_fake), axis=0)
    

    G_L = G_L1 + G_L2 + G_L3 + G_L4

    # Since the goals isn't classification, the discriminator accuracy is just to see how good the cirtic is
    # Only get accuracy among real data.
    # accuracy--This is cross validation accuracy within the training set--Nov, 2019
    #correct_prediction = tf.equal(tf.argmax(D_real_prob[:, :-1], 1),#arg max returns the indices--Bill Zhai Aug, 2019
                                  #tf.argmax(extended_label[:, :-1], 1), name='correct_prediction')
    
    # D_real_prob has shape [batch_size, 1] need to reduce the second dimension
    D_real_prob_reduced_dimension = tf.squeeze(D_real_prob, axis=1)
    correct_prediction = tf.equal(D_real_prob_reduced_dimension >= 0.5, True) # Whether the data is predicted as "real"
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return D_L, D_L_hinge_real_be_real, D_L_hinge_fake_be_fake, D_L_regression_error, G_L, G_L1, G_L2, G_L3, G_L4, correct_prediction, accuracy

In [22]:
def optimizer(D_Loss, G_Loss, D_learning_rate, G_learning_rate):
    # D and G optimizer
    extra_update_ops = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.UPDATE_OPS) # Batch_norm ops
    with tf.compat.v1.control_dependencies(extra_update_ops): # Make sure batch_norms' running mean and var are updated
        all_vars = tf.compat.v1.trainable_variables()
        D_vars = [var for var in all_vars if var.name.startswith('Discriminator')]
        G_vars = [var for var in all_vars if var.name.startswith('Generator')]
        #print('D_vars:')
        #print(D_vars)
        D_optimizer = tf.compat.v1.train.AdamOptimizer(D_learning_rate).minimize(D_Loss, var_list = D_vars)
        G_optimizer = tf.compat.v1.train.AdamOptimizer(G_learning_rate).minimize(G_Loss, var_list = G_vars)
        return D_optimizer, G_optimizer

# Visualize

In [23]:
#Assume len(data) > 5 --BZ Nov 30, 2019

# def plot_fake_data(data, epoch):
#     # visualize some data generated by G
#     data = (1/2.5) * data + 0.5
#     fig, axs = plt.subplots(len(data), figsize=(30,30))
#     cnt = 0
#     for j in range(len(data)):
#         #print(j)
#         #print(data[cnt, :, :, :])
#         axs[j].imshow(data[cnt, :, :, :])
#         axs[j].axis("off")
#         cnt = cnt + 1
#     print('graphed!')        
#     if not os.path.exists("./training_fake_figure"):
#         os.mkdir("./training_fake_figure");
#     plt.savefig("training_fake_figure/%d.jpg" % epoch)
#     plt.close()

def save_fake_image(data, epoch):
    #if not os.path.exists("./training_fake_imageMatrix"):
        #os.mkdir("./training_fake_imageMatrix");
    #np.save("./training_fake_imageMatrix/epoch_"+str(epoch), data)
    #only plot the last image of the batch
    if not os.path.exists("./training_fake_figure"):
        os.mkdir("./training_fake_figure");
    # Select the last fake image in the batch and reduce its shape
    # to 32, 32
    data_reduce = np.squeeze(data[-1], axis=2)
    plt.imshow(data_reduce)
    plt.set_cmap('hot')
    plt.axis('off')
    #print(data[-1])
    plt.savefig("training_fake_figure/%d.jpg" % epoch)
    plt.close()

In [24]:
# def save_model_on_improvement(file_path, sess, cv_acc, cv_accs):
#   #  # save model when there is improvemnet in cv_acc value
#     if cv_accs == [] or cv_acc >= np.max(cv_accs):
#         saver = tf.train.Saver(max_to_keep = 1)
#         saver.save(sess, file_path)
#         print('Model saved')
#     print('')

In [25]:
#save the top 5 best model based on validation accuracy
def save_model_top_five(folder_path, sess, cv_acc, cv_accs):
    #cv_acc is inside cv_accs
    if not os.path.exists(folder_path+'/'+task_name):
        os.mkdir(folder_path+'/'+task_name)
    sortedAccs = np.sort(cv_accs)
    for i in range(len(cv_accs)):
        if i >= 5:
            return
        if cv_acc >= sortedAccs[i]:
            saver = tf.compat.v1.train.Saver(max_to_keep = 1)
            saver.save(sess, folder_path+'/'+task_name+'/'+'_top_'+str(i+1)+'_ISING_GAN.ckpt')
            return

In [26]:
# Save model based on TPR and TNR criteria
# def save_model_TPR_TNR(folder_path, sess, epoch, cv_acc, cv_accs, TPR, TNR, TPRs):
#     if not os.path.exists(folder_path+'/'+task_name):
#         os.mkdir(folder_path+'/'+task_name)
#     sortedTPRs = np.sort(TPRs)[::-1] # Sort in Descending Order!
#     for i in range(len(sortedTPRs)):
#         if i >= 5:
#             return
#         #if TPR >= sortedTPRs[i] and TPR >= 0.88 and TNR >= 0.91:
#         if TPR >= sortedTPRs[i]:
#             print('save model')
#             saver = tf.train.Saver(max_to_keep = 1)
#             saver.save(sess, folder_path+'/'+task_name+'/'+'_TPR_top_'+str(i+1)+'_epoch_'+str(epoch)+'_SSL_GAN.ckpt')
#             return

In [27]:
# def save_model(file_path, sess):
#     saver = tf.train.Saver(max_to_keep = 1)
#     saver.save(sess, file_path)
#     print('Every 500 model saved')

In [28]:
def log_loss_acc_binary(file_path, epoch, L_D_hinge1, L_D_hinge2, L_D_reg, L_G_reg, L_G_heu, L_G_fea, L_G_avgmag, log_mode = 'a'):
    # log train and cv losses as well as accuracy
    mode = log_mode if epoch == 0 else 'a'

    with open(file_path, mode) as f:
        if mode == 'w':
            header = 'epoch, L_D_hinge1, L_D_hinge2, L_D_reg, L_G_reg, L_G_heu, L_G_fea, L_G_avgmag,' \
                     'train_Acc, val_Acc \n'
            f.write(header)

        line = '%d, %f, %f, %f, %f, %f, %f, %f\n' %(epoch, L_D_hinge1, L_D_hinge2, L_D_reg, \
                                                            L_G_reg, L_G_heu, L_G_fea, L_G_avgmag)
        f.write(line)

In [29]:
#Modified by BZ on Nov 3, 2019
#Note: predictions vector has binary entries: 1 corresponds to correct prediction, 0 is wrong prediction
def compute_val_accuracy(correct_predictions):
    return np.sum(correct_predictions)/len(correct_predictions) # or tf.reduce_mean

In [30]:
#training function--Bill Zhai Aug 2019


In [31]:
# Note: batch_size is the number of REAL data supplied to each iteration
def train_SSL_GAN(batch_size, num_epochs, train_data_by_class, train_temp_by_class, test_data_by_class, test_temp_by_class):
    tf.compat.v1.disable_eager_execution()
    # train Semi-Supervised Learning GAN
    train_D_losses, train_G_losses, train_Accs = [], [], []
    val_D_losses, val_G_losses, val_Accs, TPRs = [], [], [], []
    
    cv_size = batch_size
    num_train_exs = numTrain
    num_val_exs = numTest
    print(batch_size)
    print("num_train_exs: ", num_train_exs)
    print("num_val_exs: ", num_val_exs)

    tf.compat.v1.reset_default_graph()

    x = tf.compat.v1.placeholder(tf.float32, name = 'x', shape = [None, x_height, x_width, num_channels])
    temperature = tf.compat.v1.placeholder(tf.float32, name='temperature', shape=[None, 1])
    #label = tf.placeholder(tf.float32, name = 'label', shape = [None, num_classes]) # one hot label--BZ, August, 2019
    #labeled_mask = tf.placeholder(tf.float32, name = 'labeled_mask', shape = [None])
    z = tf.compat.v1.placeholder(tf.float32, name = 'z', shape = [None, latent_size])#one 1-d noise vector per training example
    dropout_rate = tf.compat.v1.placeholder(tf.float32, name = 'dropout_rate')
    is_training = tf.compat.v1.placeholder(tf.bool, name = 'is_training')
    G_learning_rate = tf.compat.v1.placeholder(tf.float32, name = 'G_learning_rate')
    D_learning_rate = tf.compat.v1.placeholder(tf.float32, name = 'D_learning_rate')

    model = build_model(x, z, temperature, dropout_rate, is_training, print_summary = False)
    D_real_features, D_real_logit, D_real_prob, D_real_regression, \
    D_fake_features, D_fake_logit, D_fake_prob, D_fake_regression, fake_data = model
    #extended_label = prepare_labels(label) #is only for real image data
        
    loss_acc  = loss_accuracy(x, fake_data, D_real_features, D_real_logit, D_real_prob, D_real_regression,
                              D_fake_features, D_fake_logit, D_fake_prob, D_fake_regression, temperature)

    D_L, D_L_hinge_1, D_L_hinge_2, D_L_regression_error, G_L, G_L1, G_L2, G_L3, G_L4, correct_prediction, accuracy = loss_acc
    D_optimizer, G_optimizer = optimizer(D_L, G_L, G_learning_rate, D_learning_rate)

    
#     validation_generator = get_batch(data_path, label_path, num_val_exs, num_train_exs, True)

    print('training....')

    with tf.compat.v1.Session() as sess:       
        sess.run(tf.compat.v1.global_variables_initializer())

        #t_total = 0
        for epoch in range(num_epochs):
            #for iteration in range(int(numTrain/(batch_size*(num_classes+c_ul)/(num_classes+1+c_ul)))):
            #batch_num = 0 #added--BZ Nov 2, 2019
            #training_generator = get_training_batch_and_labeled_mask(X_train, y_train, LABELED_MASK, batch_size);
            train_batch_generator = get_train_batch(train_data_by_class,
                                                                  train_temp_by_class,
                                                                  batch_size)
            for train_batch_x, train_batch_temp in train_batch_generator:
                #train_batch_x, train_batch_y, train_batch_mask = get_balance_train_batch_3(train_data_by_class, train_label_by_class, train_mask_by_class, batch_size, c_ul)            #for train_batch_x, train_batch_y, train_batch_mask in training_generator:
                #t_start = time.time()
                #batch_z = np.random.uniform(-1.0, 1.0, size = (batch_size, latent_size)) #
                #batch_z = np.random.normal(0, 1, size = (int(batch_size/num_classes), latent_size))
                batch_z = np.random.normal(0, 1, size = [len(train_batch_temp), latent_size])
                #function is to be modified as labeled mask should stay with "labeled" images which are shuffled BZ--August, 2019
                #mask = get_training_batch_and_labeled_mask(XTrain, yTrain, labeledMask, batchSize);--marked as solved BZ, Nov, 2019
                train_feed_dictionary = {x: train_batch_x,
                                         z: batch_z,
                                         temperature: train_batch_temp,
                                         #labeled_mask: train_batch_mask,
                                         dropout_rate: 0.25,
                                         G_learning_rate: 2e-4,
                                         D_learning_rate: 2e-4,
                                         is_training: True} # BatchNorm training mode
                
                fetch_ops =sess.run(fetches=[D_optimizer, G_optimizer, D_L, G_L, accuracy, 
                                             D_L_hinge_1, D_L_hinge_2, D_L_regression_error, G_L1, G_L2, G_L3, G_L4], 
                                    feed_dict=train_feed_dictionary)
                _, _, train_D_loss, train_G_loss, train_accuracy, \
                train_D_hinge_1, train_D_hinge_2, train_D_regression_error, train_G_L1, train_G_L2, train_G_L3, train_G_L4 = fetch_ops
                print('train_D_hinge_1, train_D_hinge_2, train_D_regression_error')
                print(train_D_hinge_1)
                print(train_D_hinge_2)
                print(train_D_regression_error)
                print('\n')
                print("train_G_L1, train_G_L2, train_G_L3, train_G_l4")
                print(train_G_L1)
                print(train_G_L2)
                print(train_G_L3)
                print(train_G_L4)
                print('==================')
                #t_total += (time.time() - t_start)

                #debug:
                #trainDLogits = D_real_logit.eval(feed_dict = train_feed_dictionary)
                #trainRealFeatures = D_real_features.eval(feed_dict = train_feed_dictionary)
            
                
                
                #print('Epoch: '+str(epoch)+" Iter: "+str(iteration)+" Time: "+str(t_total)+" train_G_loss: "
                    #  +str(train_G_loss)+" train_D_loss: "+str(train_D_loss)+" train_accuracy: "+str(train_accuracy))

                #print('trainDLogits:')
                #print(trainDLogits)
                #print('trainFeatures:')
                #print(trainRealFeatures)
                #print('xinput')
                #print(xinput.eval(feed_dict = train_feed_dictionary))
                #print('dropout1')
                #print(dropout1.eval(feed_dict = train_feed_dictionary))
                #print('batch_norm2')
                #print(batch_norm2.eval(feed_dict = train_feed_dictionary))
                #print('dropout3')
                #print(dropout3.eval(feed_dict = train_feed_dictionary))
                #print('dropout4')
                #print(dropout4.eval(feed_dict = train_feed_dictionary))
        
                train_D_losses.append(train_D_loss)
                train_G_losses.append(train_G_loss)
                train_Accs.append(train_accuracy)
                #batch_num = batch_num + 1;
            # Validation at the end of each epoch--BZ, Nov, 2019

            
            test_generator= get_test_batch(test_data_by_class, test_temp_by_class, batch_size)
            val_correct_preds = []
            val_real_probs = np.zeros((0,1))
            #labels = np.zeros((0, num_classes))
            for test_batch_x, test_batch_temp in test_generator:
            #test_batch_generator = get_test_batch(X_test, y_test, batch_size);#added--BZ, Nov 2, 2019
            #for test_batch_x, test_batch_y in test_batch_generator:
                val_batch_z = np.random.normal(0, 1, size = [len(test_batch_temp), latent_size])
                #mask = np.ones(len(test_batch_y));#all test data is labeled   added--BZ, Nov 2, 2019
                val_feed_dictionary = {x: test_batch_x,
                                       z: val_batch_z,
                                       #label: test_batch_y,
                                       #labeled_mask: mask,
                                       temperature: test_batch_temp,
                                       dropout_rate: 0.0,
                                       is_training: False} # BatchNorm training mode


                #val_D_loss = D_L.eval(feed_dict = val_feed_dictionary)
                #val_G_loss = G_L.eval(feed_dict = val_feed_dictionary)

                #val_correct_pred = correct_prediction.eval(feed_dict = val_feed_dictionary)
                
                    
                #val_D_real_prob = D_real_prob.eval(feed_dict = val_feed_dictionary)
                
                fetch_ops = sess.run(fetches=[D_L_hinge_1, D_L_hinge_2, D_L_regression_error, G_L1, G_L2, G_L3, G_L4],
                                    feed_dict=val_feed_dictionary)
                #val_D_loss, val_G_loss, val_correct_pred, val_D_real_prob = fetch_ops
                L_D_hinge1, L_D_hinge2, L_D_reg,\
                L_G_heu, L_G_reg, L_G_fea, L_G_avgmag = fetch_ops
                
                #val_correct_preds = np.concatenate((val_correct_preds, val_correct_pred))
                #val_real_probs = np.concatenate((val_real_probs, val_D_real_prob))
                #val_fake_probs = np.concatenate((val_fake_probs, val_fake_prob))
                #predictions = np.concatenate((predictions, np.argmax(val_D_real_prob[:, :-1], axis = 1)))
                #labels = np.concatenate((labels, test_batch_y))
            
            #val_accuracy = compute_val_accuracy(val_correct_preds)
            #val_Accs.append(val_accuracy)
    
            #CM = confusion_matrix(np.argmax(labels, axis = 1), predictions)
            #TPR = CM[1][1]/(CM[1][0]+CM[1][1])
            #TNR = CM[0][0]/(CM[0][0]+CM[0][1])
            #TPRs.append(TPR)
            #print(val_correct_preds);
            #print('validation_acc: %f' %(val_accuracy))
            log_loss_acc_binary(log_path, epoch, L_D_hinge1, L_D_hinge2, L_D_reg, L_G_reg, L_G_heu, L_G_fea, L_G_avgmag, 'w')
    
            #save_model_top_five(model_path, sess, val_accuracy, val_Accs)
            
            #fakes = fake_data.eval(feed_dict = val_feed_dictionary)
            #save_fake_image(fakes, epoch)
            #print(fakes)
            #confusion matrix
            #print('epoch'+str(epoch)+' CM:')
            #print(confusion_matrix(np.argmax(labels, axis = 1), predictions, normalize = 'true'))
            #print('val_real_probs: ')
            #print(val_real_probs)
            #print('val_fake_probs: ')
            #print(val_fake_probs)
            #print('epoch '+str(epoch)+' done')  
            
        
    return train_D_losses, train_G_losses, train_Accs, val_Accs

In [32]:
train_SSL_GAN(60, 300, train_data_by_class, train_temp_by_class, test_data_by_class, test_temp_by_class)

60
num_train_exs:  18000
num_val_exs:  2000
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
output_classification_prob.get_shape() is
(None, 1)
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
output_classification_prob.get_shape() is
(None, 1)
real_imgs
batch_size is
Tensor("strided_slice:0", shape=(), dtype=int32)
training....
Device mapping:

train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[1.4019425]
[0.6579832]
[4.5441976]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[1.1451783]
[2.3440633]
0.107082605
0.954871
train_D_hinge_1, train_D_hinge_2, train_D_regressi

train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.02235393]
[0.00302302]
[0.9966619]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.723236]
[1.2192589]
0.011834894
0.8116141
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.02074152]
[0.01005482]
[1.402995]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.5553107]
[1.1751969]
0.011684487
0.7967346
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.02935871]
[0.01039425]
[1.321344]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.683285]
[1.2471212]
0.011521509
0.7881809
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.04940693]
[-0.]
[0.8967588]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.3492203]
[0.81378216]
0.011272431
0.79205984
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.01764287]
[0.00137359]
[1.4567142]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.3432233]
[0.8500598]
0.011350011
0.78608984
train_D_hinge_1, train_D_hinge_2, train_D_regres

train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.01557799]
[0.01119963]
[1.4854928]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.3350089]
[0.79597956]
0.009513334
0.60762584
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.0051229]
[0.0008236]
[1.0920354]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.5731654]
[0.88318187]
0.0095277345
0.6035407
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00589832]
[0.00387863]
[0.89467126]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.654772]
[0.7956295]
0.0096385125
0.60179263
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00703672]
[0.00621196]
[1.2791271]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.474452]
[0.8077654]
0.009648621
0.5978123
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00511343]
[0.00926579]
[1.0215491]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.4406047]
[0.6262759]
0.009510417
0.5826878
train_D_hinge_1, train_D_hinge_2, tra

train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00391145]
[0.0061409]
[0.78457206]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.7836318]
[0.9401627]
0.012226889
0.55432236
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00324234]
[0.01174314]
[0.82394636]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.5479937]
[0.57625663]
0.012331149
0.55482036
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00590831]
[0.00287967]
[0.8662271]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.6006556]
[0.69697267]
0.012285909
0.5495603
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00458758]
[-0.]
[0.990285]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.8011267]
[0.9203675]
0.012749681
0.5531708
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.0086092]
[-0.]
[0.91097695]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.8635492]
[0.66760576]
0.012574156
0.55516213
train_D_hinge_1, train_D_hinge_2, train_D_regres

train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00295959]
[-0.]
[0.9645701]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.968072]
[1.0571542]
0.014780772
0.5788701
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00180296]
[-0.]
[0.8630895]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.959858]
[0.6531837]
0.015055724
0.5960092
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00388567]
[-0.]
[0.8520301]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[2.9979596]
[0.7780092]
0.015196101
0.5918737
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[-0.]
[-0.]
[1.1172833]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.0251508]
[0.6126184]
0.015435646
0.60004383
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[-0.]
[0.00100384]
[0.8933975]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.0630367]
[0.8815021]
0.015274143
0.5979752
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00744242]
[-0.]
[0.60

train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00202863]
[-0.]
[0.64020145]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.338004]
[0.78993237]
0.01884361
0.62261987
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00068915]
[0.00073511]
[0.7717432]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.6947808]
[0.8100204]
0.018974986
0.62054163
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00201248]
[-0.]
[0.74466896]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.2207682]
[0.84151524]
0.019234393
0.6167719
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[-0.]
[-0.]
[0.8294148]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.3348536]
[0.6128019]
0.019261345
0.6183261
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[-0.]
[0.00221104]
[1.0419856]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.4707255]
[0.6533638]
0.019045545
0.598795
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[-0.]
[-0.]
[

train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[-0.]
[-0.]
[0.64519244]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.439189]
[0.8006694]
0.025554912
0.6297022
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[-0.]
[-0.]
[0.7471945]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.7417717]
[0.7475676]
0.02510841
0.6286104
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[-0.]
[-0.]
[0.9260817]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.6158285]
[0.6968082]
0.025227822
0.6322796
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[-0.]
[-0.]
[0.84752464]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.6076808]
[0.6325418]
0.025232337
0.62516224
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[-0.]
[-0.]
[0.8093141]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.5177932]
[0.78231776]
0.025540499
0.6343815
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[-0.]
[0.00350126]
[1.0606248]


train_G_L1, trai

train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[-0.]
[-0.]
[0.837558]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.8225977]
[0.72075284]
0.030338727
0.6288843
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00030649]
[-0.]
[0.80749136]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.7907195]
[0.65674424]
0.030176483
0.6296865
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.0015788]
[-0.]
[0.7719813]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.748906]
[0.7443603]
0.03094076
0.6335649
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[-0.]
[-0.]
[0.6768618]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.8976717]
[0.7148191]
0.030321818
0.63357085
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[0.00334187]
[-0.]
[0.75396955]


train_G_L1, train_G_L2, train_G_L3, train_G_l4
[3.8803067]
[0.667801]
0.030715134
0.6352282
train_D_hinge_1, train_D_hinge_2, train_D_regression_error
[-0.]
[-0.]
[0.78411424]


tra

KeyboardInterrupt: 

In [ ]:
test_temp_by_class[0]

In [ ]:
def confusion_matrix(predictions, labels, numClasses):
            #pred 0   #pred 1
    #real 0
    #real 1
    matrix = np.zeros((numClasses, numClasses))
    for n in range(len(predictions)):
        matrix[int(labels[n]), int(predictions[n])] += 1
    return matrix
        

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
#Retrieve best DCGAN model and run the whole training set or test set to get the feature map vectors, 
#which are later used to either
#i. train a Random Forest Model
#or
#ii. run on test set
#start new session:
def testModel(test_data_by_class, test_label_by_class, batch_size):
    labels = np.zeros((0,num_classes))
    predictions = np.zeros((0))
    correct_preds = np.zeros((0))
    
    tf.reset_default_graph()
    with tf.Session() as sess: 
        #bestModel = tf.train.import_meta_graph('./savedModels/GAN/'+str(datetime.date.today())+'_SSL_GAN.ckpt.meta')#load graph
        bestModel = tf.train.import_meta_graph('./savedModels/GAN/2020-01-27_SSL_GAN.ckpt.meta')#load graph

        bestModel.restore(sess,tf.train.latest_checkpoint('./savedModels/GAN'))#load parameters
        #feed placeholders:
        x = sess.graph.get_tensor_by_name("x:0")
        label = sess.graph.get_tensor_by_name("label:0")
        labeled_mask = sess.graph.get_tensor_by_name("labeled_mask:0")
        dropout_rate = sess.graph.get_tensor_by_name("dropout_rate:0")
        is_training = sess.graph.get_tensor_by_name("is_training:0")
        #output metrics of interest:
        D_real_prob = sess.graph.get_tensor_by_name("Discriminator/D_output:0")
        G_output = sess.graph.get_tensor_by_name("Generator/G_output:0")
        correct_prediction = sess.graph.get_tensor_by_name("correct_prediction:0")
#         prediction = tf.argmax(D_real_prob, axis = 1)
        prediction = tf.argmax( D_real_prob[:, :-1], axis = 1)

        test_generator= get_test_batch(test_data_by_class, test_label_by_class, batch_size)
        
        #for debugging: output weight names
        weights = [v.eval(session=sess) for v in tf.trainable_variables()]
        weightNames = [v for v in tf.trainable_variables()]
        i = 0
        for batch_x, batch_y in test_generator:
            #print('batch_y:')
            #print(batch_y)
            mask = np.ones(len(batch_y));#all test data is labeled   added--BZ, Nov 2, 2019
            feed_dictionary = {x: normalize(batch_x),
                               label: batch_y,
                               labeled_mask: mask,
                               dropout_rate: 0.0,
                               is_training: False}
            
            #accuracy_eval = accuracy.eval(feed_dict = feed_dictionary)
            correct_pred_eval = correct_prediction.eval(feed_dict = feed_dictionary)
            D_real_prob_eval = D_real_prob.eval(feed_dict = feed_dictionary)
            
            correct_preds = np.concatenate((correct_preds, correct_pred_eval))
            prediction_eval = prediction.eval(feed_dict=feed_dictionary)
            labels = np.concatenate((labels, batch_y))#array of k+1 one hot
            predictions = np.concatenate((predictions, prediction_eval))#array of ints
            #print('real features: batch'+str(i))
            #print(D_real_features_eval)
            #print('accuracy: '+str(accuracy_eval))
            #print(str(D_real_features_eval.shape))
            #print('labels'+str(labels.shape))
            #print('featuremaps'+str(feature_maps.shape))
            #print('real_prob:')
            #print(D_real_prob_eval)
            #print('dropout1:')
            #print(dropout1.eval(feed_dict = feed_dictionary))
            #print('conv1:')
            #print(conv1.eval(feed_dict = feed_dictionary))
            #print('x:')
            #print(x.eval(feed_dict = feed_dictionary))
            #print('D_real_prob:')
            #print(D_real_prob_eval)
            i = i + 1
        sess.close()
    total_accuracy = np.sum(correct_preds)/len(correct_preds)
    print('total accuracy: '+str(total_accuracy))
#     print('labels:')
#     print(labels)
    print('predictions:')
    print(predictions)
    print('Confusion Matirx:')
#     print(confusion_matrix(predictions, np.argmax(labels, axis = 1), num_classes+1))
    print(confusion_matrix(np.argmax(labels, axis = 1), predictions))
#     return labels, weights, weightNames
    return predictions, labels, weights, weightNames





    

    

In [ ]:
#Test model:
predictions, labels_test, params, paramNames = testModel(test_data_by_class, test_label_by_class, 100)

In [ ]:
# Baseline Model
def loss_accuracy_baseline(D_real_features, D_real_logit, D_real_prob, extended_label, labeled_mask):
    epsilon = 1e-8 # used to avoid NAN loss
    # *** Discriminator loss ***
    # supervised loss
    # which class the real data belongs to
    tmp = tf.nn.softmax_cross_entropy_with_logits(logits = D_real_logit,#cross_entrypy_with_logits is the only function available
                                                  labels = extended_label)
    #question: is this an over simplification?--no, becuase tmp is a vector: num_samples_in_batch * 1
    D_L_supervised = tf.reduce_sum(labeled_mask * tmp) / (tf.reduce_sum(labeled_mask)+1e-6) # to ignore unlabeled data
                                                                                     
    # accuracy--This is cross validation accuracy within the training set--Nov, 2019
    correct_prediction = tf.equal(tf.argmax(D_real_prob[:, :-1], 1),#arg max returns the indices--Bill Zhai Aug, 2019
                                  tf.argmax(extended_label[:, :-1], 1), name='correct_prediction')
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #cast boolean to float32--Bill Zhai Aug, 2019
    
    
    return D_L_supervised, accuracy, correct_prediction 

In [ ]:
def optimizer_baseline(D_Loss, D_learning_rate):
    # D and G optimizer
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        all_vars = tf.trainable_variables()
        D_vars = [var for var in all_vars if var.name.startswith('Discriminator')]
        D_optimizer = tf.train.AdamOptimizer(D_learning_rate).minimize(D_Loss, var_list = D_vars)
        return D_optimizer

In [ ]:
#build model for each batch using D() and G() functions
def build_model_baseline(x_real, label, dropout_rate, is_training, print_summary = False):
    # build model
    #Discriminator on real data (labeled and unlabeled)  flatten5, fc5, output
    D_real_features, D_real_logit, D_real_prob = D(x_real, dropout_rate, is_training,
                                                   reuse = False, print_summary = print_summary)
    return D_real_features, D_real_logit, D_real_prob

In [ ]:
def train_baseline(batch_size, num_epochs, train_data_by_class, train_label_by_class, test_data_by_class, test_label_by_class):
    # train Semi-Supervised Learning GAN
    train_D_losses, train_Accs = [], []
    val_D_losses, val_Accs = [], []

    cv_size = batch_size
    num_train_exs = numTrain
    num_val_exs = numTest
    print(batch_size)
    print("num_train_exs: ", num_train_exs)
    print("num_val_exs: ", num_val_exs)

    tf.reset_default_graph()

    x = tf.placeholder(tf.float32, name = 'x', shape = [None, x_height, x_width, num_channels])
    label = tf.placeholder(tf.float32, name = 'label', shape = [None, num_classes]) # one hot label--BZ, August, 2019
    labeled_mask = tf.placeholder(tf.float32, name = 'labeled_mask', shape = [None])
    dropout_rate = tf.placeholder(tf.float32, name = 'dropout_rate')
    is_training = tf.placeholder(tf.bool, name = 'is_training')
    D_learning_rate = tf.placeholder(tf.float32, name = 'D_learning_rate')

    model = build_model_baseline(x, label, dropout_rate, is_training, print_summary = False)
    D_real_features, D_real_logit, D_real_prob = model
    extended_label = prepare_labels(label) #is only for real image data
    loss_acc  = loss_accuracy_baseline(D_real_features, D_real_logit, D_real_prob,
                              extended_label, labeled_mask)
    D_L, accuracy, correct_prediction = loss_acc
    D_optimizer = optimizer_baseline(D_L, D_learning_rate)

    
#     validation_generator = get_batch(data_path, label_path, num_val_exs, num_train_exs, True)

    print('training....')

    with tf.Session() as sess:       
        sess.run(tf.global_variables_initializer())
        #mnist_set = get_data()

        t_total = 0
        #changed to iterating on number of epochs!
        iter_count = 0
        iter_since_last_val = 0
        for epoch in range(num_epochs):
            for iteration in range(int(numTrain/batch_size)):
            #batch_num = 0 #added--BZ Nov 2, 2019
            #training_generator = get_training_batch_and_labeled_mask(X_train, y_train, LABELED_MASK, batch_size);
                train_batch_x, train_batch_y, train_batch_mask = get_balance_train_batch(train_data_by_class, train_label_by_class, train_mask_by_class, batch_size)
            #for train_batch_x, train_batch_y, train_batch_mask in training_generator:
                t_start = time.time()
                #function is to be modified as labeled mask should stay with "labeled" images which are shuffled BZ--August, 2019
                #mask = get_training_batch_and_labeled_mask(XTrain, yTrain, labeledMask, batchSize);--marked as solved BZ, Nov, 2019
                train_feed_dictionary = {x:  normalize(train_batch_x),
                                         label: train_batch_y,
                                         labeled_mask: train_batch_mask,
                                         dropout_rate: 0.0,
                                         D_learning_rate: 1e-5,
                                         is_training: True}

                D_optimizer.run(feed_dict = train_feed_dictionary)

                train_D_loss = D_L.eval(feed_dict = train_feed_dictionary)
                train_accuracy = accuracy.eval(feed_dict = train_feed_dictionary)
                t_total += (time.time() - t_start)

                print('Epoch: '+str(epoch)+" Iter: "+str(iteration)+" Time: "+str(t_total)+" train_D_loss: "+str(train_D_loss)+" train_accuracy: "+str(train_accuracy))

                #print('trainDLogits:')
                #print(trainDLogits)
                #print('trainFeatures:')
                #print(trainRealFeatures)
                #print('xinput')
                #print(xinput.eval(feed_dict = train_feed_dictionary))
                #print('dropout1')
                #print(dropout1.eval(feed_dict = train_feed_dictionary))
                #print('batch_norm2')
                #print(batch_norm2.eval(feed_dict = train_feed_dictionary))
                #print('dropout3')
                #print(dropout3.eval(feed_dict = train_feed_dictionary))
                #print('dropout4')
                #print(dropout4.eval(feed_dict = train_feed_dictionary))

                train_D_losses.append(train_D_loss)
                train_Accs.append(train_accuracy)
                #batch_num = batch_num + 1;
            # Validation at the end of each epoch--BZ, Nov, 2019

            
            test_generator= get_test_batch(test_data_by_class, test_label_by_class, batch_size)
            val_correct_preds = []
            for test_batch_x, test_batch_y in test_generator:
            #test_batch_generator = get_test_batch(X_test, y_test, batch_size);#added--BZ, Nov 2, 2019
            #for test_batch_x, test_batch_y in test_batch_generator:
                mask = np.ones(len(test_batch_y));#all test data is labeled   added--BZ, Nov 2, 2019
                val_feed_dictionary = {x: normalize(test_batch_x),
                                       label: test_batch_y,
                                       labeled_mask: mask,
                                       dropout_rate: 0.0,
                                       is_training: False}


                val_D_loss = D_L.eval(feed_dict = val_feed_dictionary)

                val_correct_pred = correct_prediction.eval(feed_dict = val_feed_dictionary)
                val_correct_preds = np.concatenate((val_correct_preds, val_correct_pred))
                    
            
            val_accuracy = compute_val_accuracy(val_correct_preds)
            val_Accs.append(val_accuracy)
    
            #print(val_correct_preds);
            #print('validation_acc: %f' %(val_accuracy))
            log_loss_acc('./baseline_log_'+str(datetime.date.today())+'.csv', epoch, train_D_loss, 0, train_accuracy,
                 val_accuracy, 'w')
    
            save_model_on_improvement(baseline_path+'/'+str(datetime.date.today())+'_baseline.ckpt', sess, val_accuracy, val_Accs)
            
                
        
    return train_D_losses, train_Accs, val_Accs

In [ ]:
train_baseline(64, 500000, train_data_by_class, train_label_by_class, test_data_by_class, test_label_by_class)

In [ ]:
#Retrieve best DCGAN model and run the whole training set or test set to get the feature map vectors, 
#which are later used to either
#i. train a Random Forest Model
#or
#ii. run on test set
#start new session:
def testModel_baseline(test_data_by_class, test_label_by_class, batch_size):
    labels = np.zeros((0,num_classes))
    predictions = np.zeros((0))
    correct_preds = np.zeros((0))
    
    tf.reset_default_graph()
    with tf.Session() as sess: 
        bestModel = tf.train.import_meta_graph('./savedModels/baseline/'+str(datetime.date.today())+'_baseline.ckpt.meta')#load graph
        bestModel.restore(sess,tf.train.latest_checkpoint('./savedModels/baseline'))#load parameters
        #feed placeholders:
        x = sess.graph.get_tensor_by_name("x:0")
        label = sess.graph.get_tensor_by_name("label:0")
        labeled_mask = sess.graph.get_tensor_by_name("labeled_mask:0")
        dropout_rate = sess.graph.get_tensor_by_name("dropout_rate:0")
        is_training = sess.graph.get_tensor_by_name("is_training:0")
        #output metrics of interest:
        D_real_prob = sess.graph.get_tensor_by_name("Discriminator/D_output:0")
        correct_prediction = sess.graph.get_tensor_by_name("correct_prediction:0")
#         prediction = tf.argmax(D_real_prob, axis = 1)
        prediction = tf.argmax( D_real_prob[:, :-1], axis = 1)

        test_generator= get_test_batch(test_data_by_class, test_label_by_class, batch_size)
        
        #for debugging: output weight names
        weights = [v.eval(session=sess) for v in tf.trainable_variables()]
        weightNames = [v for v in tf.trainable_variables()]
        i = 0
        for batch_x, batch_y in test_generator:
            #print('batch_y:')
            #print(batch_y)
            mask = np.ones(len(batch_y));#all test data is labeled   added--BZ, Nov 2, 2019
            feed_dictionary = {x: normalize(batch_x),
                               label: batch_y,
                               labeled_mask: mask,
                               dropout_rate: 0.0,
                               is_training: False}
            
            #accuracy_eval = accuracy.eval(feed_dict = feed_dictionary)
            correct_pred_eval = correct_prediction.eval(feed_dict = feed_dictionary)
            D_real_prob_eval = D_real_prob.eval(feed_dict = feed_dictionary)
            
            correct_preds = np.concatenate((correct_preds, correct_pred_eval))
            prediction_eval = prediction.eval(feed_dict=feed_dictionary)
            labels = np.concatenate((labels, batch_y))#array of k+1 one hot
            predictions = np.concatenate((predictions, prediction_eval))#array of ints
            #print('real features: batch'+str(i))
            #print(D_real_features_eval)
            #print('accuracy: '+str(accuracy_eval))
            #print(str(D_real_features_eval.shape))
            #print('labels'+str(labels.shape))
            #print('featuremaps'+str(feature_maps.shape))
            #print('real_prob:')
            #print(D_real_prob_eval)
            #print('dropout1:')
            #print(dropout1.eval(feed_dict = feed_dictionary))
            #print('conv1:')
            #print(conv1.eval(feed_dict = feed_dictionary))
            #print('x:')
            #print(x.eval(feed_dict = feed_dictionary))
            #print('D_real_prob:')
            #print(D_real_prob_eval)
            i = i + 1
        sess.close()
    total_accuracy = np.sum(correct_preds)/len(correct_preds)
    print('total accuracy: '+str(total_accuracy))
#     print('labels:')
#     print(labels)
    print('predictions:')
    print(predictions)
    print('Confusion Matirx:')
#     print(confusion_matrix(predictions, np.argmax(labels, axis = 1), num_classes+1))
    print(confusion_matrix(np.argmax(labels, axis = 1), predictions))
#     return labels, weights, weightNames
    return predictions, labels, weights, weightNames



In [ ]:
testModel_baseline(test_data_by_class, test_label_by_class, 100)